<a href="https://colab.research.google.com/github/j7tfj7f8k1f/DL-Fine-Tuning/blob/main/BLIP2_model_LORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 以下是LORA的Fine Tuning BLIP2模型的方式

## 前置作業HF_TOKEN = huggingface的金鑰





# 要達成提問搜尋圖片搜尋的BLIP2模型需要一張圖搭配一組描述詞
# 比方睡衣吉伊卡哇.png + txt檔案內容為
# 裝飾整體是穿著粉紅條紋睡衣和頭戴粉色兔耳朵頭飾配有棕色小熊 身體配件右手配件是粉色毛線球 表情開心 毛兩撇 嘴型微笑
#它是睡衣吉伊卡哇

# 不能有結巴 BLIP2模型需要學習我們人類的句子用法

# 有加入反樣本結果比較好 我們採用排除自己 剩下隨機抽選

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets
!pip install -U bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00


In [ ]:
import shutil

# 定義來源和目標資料夾路徑
source_folder = '/content/'
destination_folder = '/content/'  # 使用不同的名稱避免覆蓋

# 複製資料夾
shutil.copytree(source_folder, destination_folder)

print(f"已將 '{source_folder}' 複製到 '{destination_folder}'")

已將 '/content/drive/MyDrive/完整' 複製到 '/content/完整_copy'


# <<步驟1>> 建立反樣本

In [ ]:
import os
import shutil
import pandas as pd
import random
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
import time

In [ ]:
# 1. 準備資料集
# 定義路徑
root_dir = '/content/'  # 資料集根目錄
output_dir = '/content/'  # 輸出資料集目錄

# 建立輸出目錄
os.makedirs(output_dir, exist_ok=True)

# 處理資料集並建立 metadata.csv
metadata = pd.DataFrame(columns=['file_name', 'additional_feature', 'label'])
data_to_append = []

# 收集所有的文本文件內容供反樣本使用
all_text_files = []

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            if file.endswith('.txt'):
                with open(os.path.join(subdir_path, file), 'r', encoding='utf-8') as f:
                    all_text_files.append(f.read().strip())

# 遍歷子資料夾，建立正樣本和反樣本
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    print(f"Processing subdirectory: {subdir_path}")

    if os.path.isdir(subdir_path):
        image_file = None
        text = None

        # 搜尋圖像和正樣本文本
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)

            if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_file = file
                dest_path = os.path.join(output_dir, image_file)
                if not os.path.exists(dest_path):
                    shutil.copy(file_path, output_dir)
                    print(f"Copied {file} to {output_dir}")
                else:
                    print(f"Image {file} already exists in {output_dir}, skipping copy.")

            elif file.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()

        # 建立正樣本
        if image_file and text:
            data_to_append.append({'file_name': image_file, 'additional_feature': text, 'label': 'positive'})

            # 隨機選取反樣本
            negative_text = random.choice([t for t in all_text_files if t != text])
            data_to_append.append({'file_name': image_file, 'additional_feature': negative_text, 'label': 'negative'})

metadata = pd.concat([metadata, pd.DataFrame(data_to_append)], ignore_index=True)
metadata.to_csv(f'{output_dir}/metadata.csv', index=False, encoding='utf-8')
print("Images, text files, and negative samples processed and metadata saved successfully!")

## <<步驟2>>訓練模型

In [ ]:
# 使用 Hugging Face 的 ImageFolder dataset
dataset = load_dataset("imagefolder", data_dir='/content/', split="train")

# 2. 定義 Dataset 和 DataLoader
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["additional_feature"]
        return encoding

def collate_fn(batch):
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

# 3. 初始化模型和訓練參數
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)

# 定義 LoraConfig 和 get_peft_model 來微調模型
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# 建立訓練資料集和 DataLoader
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collate_fn)

# 4. 開始訓練
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"


# 建立模型儲存路徑
folder_path = '/content/drive/MyDrive/BLIP2_model'
os.makedirs(folder_path, exist_ok=True)
save_path = '/content/drive/MyDrive/BLIP2_model/best_blip2_model.pth'

# 訓練迴圈
num_epochs = 30
patience = 5  # 早停參數
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    start_time = time.time()
    epoch_loss = 0.0
    model.train()

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_epoch_loss = epoch_loss / len(train_dataloader)

    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), save_path)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss improved to {avg_epoch_loss:.4f}, saving model.")
    else:
        epochs_without_improvement += 1
        print(f"Epoch {epoch+1}/{num_epochs}, Loss did not improve. Consecutive epochs without improvement: {epochs_without_improvement}")

    if epochs_without_improvement >= patience:
        print("Early stopping triggered.")
        break

    end_time = time.time()
    elapsed_time = end_time - start_time
    remaining_time = elapsed_time * (num_epochs - epoch - 1)
    print(f"Epoch {epoch+1} completed in {elapsed_time:.2f}s, estimated time remaining: {remaining_time:.2f}s")

print("Training complete.")


Processing subdirectory: /content/完整_copy/茄子吉伊卡哇
Copied 茄子吉伊卡哇.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/墮天使小八
Copied 墮天使小八.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/伏見稻荷烏薩奇
Copied 伏見稻荷烏薩奇.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/吊飾睡衣派對栗子饅頭
Copied 吊飾睡衣派對栗子饅頭.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/掛件狼人小八
Copied 掛件狼人小八.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/吊飾中華飯店獅薩
Copied 吊飾中華飯店獅薩.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/開車小八
Copied 開車小八.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/開車獺師
Copied 開車獺師.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/魔法少女小八
Copied 魔法少女小八.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/掛件聖誕老人小八
Copied 掛件聖誕老人小八.png to /content/chiikawa/train
Processing subdirectory: /content/完整_copy/掛件平頭小八
Cop

Resolving data files:   0%|          | 0/298 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/997M [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/782M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

trainable params: 2,621,440 || all params: 3,747,301,376 || trainable%: 0.0700
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/30, Loss improved to 3.5147, saving model.
Epoch 1 completed in 180.10s, estimated time remaining: 5222.81s
Epoch 2/30, Loss improved to 2.2667, saving model.
Epoch 2 completed in 200.62s, estimated time remaining: 5617.49s
Epoch 3/30, Loss improved to 1.8068, saving model.
Epoch 3 completed in 200.52s, estimated time remaining: 5414.09s
Epoch 4/30, Loss improved to 1.5443, saving model.
Epoch 4 completed in 203.51s, estimated time remaining: 5291.35s
Epoch 5/30, Loss improved to 1.3867, saving model.
Epoch 5 completed in 199.37s, estimated time remaining: 4984.32s
Epoch 6/30, Loss improved to 1.2705, saving model.
Epoch 6 completed in 203.17s, estimated time remaining: 4875.99s
Epoch 7/30, Loss improved to 1.1909, saving model.
Epoch 7 completed in 217.17s, estimated time 

# 不滿意繼續訓練

In [ ]:
import os
import shutil
import pandas as pd
import random
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
import time

# 1. 準備資料集
root_dir = '/content/'
output_dir = '/content/'
os.makedirs(output_dir, exist_ok=True)

# 建立 metadata.csv
metadata = pd.DataFrame(columns=['file_name', 'additional_feature', 'label'])
data_to_append = []
all_text_files = []

# 收集所有文本
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for file in os.listdir(subdir_path):
            if file.endswith('.txt'):
                with open(os.path.join(subdir_path, file), 'r', encoding='utf-8') as f:
                    all_text_files.append(f.read().strip())

# 建立正樣本和反樣本
for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        image_file = None
        text = None
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_file = file
                dest_path = os.path.join(output_dir, image_file)
                if not os.path.exists(dest_path):
                    shutil.copy(file_path, output_dir)
            elif file.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()

        if image_file and text:
            data_to_append.append({'file_name': image_file, 'additional_feature': text, 'label': 'positive'})
            negative_text = random.choice([t for t in all_text_files if t != text])
            data_to_append.append({'file_name': image_file, 'additional_feature': negative_text, 'label': 'negative'})

metadata = pd.concat([metadata, pd.DataFrame(data_to_append)], ignore_index=True)
metadata.to_csv(f'{output_dir}/metadata.csv', index=False, encoding='utf-8')

# 使用 Hugging Face 的 ImageFolder dataset
dataset = load_dataset("imagefolder", data_dir='/content/', split="train")

# 2. 定義 Dataset 和 DataLoader
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["additional_feature"]
        return encoding

def collate_fn(batch):
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

# 3. 初始化模型和訓練參數
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)

# 定義 LoraConfig 和 get_peft_model 來微調模型
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

# 建立訓練資料集和 DataLoader
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collate_fn)

# 掛載 Google Drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/BLIP2_model'
os.makedirs(folder_path, exist_ok=True)
save_path = os.path.join(folder_path, 'best_blip2_model.pth')

# 載入已保存的模型權重（如果存在）
if os.path.exists(save_path):
    model.load_state_dict(torch.load(save_path))
    print("已載入之前的模型權重，繼續訓練。")

# 訓練迴圈
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

num_epochs = 15
patience = 5
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    start_time = time.time()
    epoch_loss = 0.0
    model.train()

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_epoch_loss = epoch_loss / len(train_dataloader)

    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), save_path)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss improved to {avg_epoch_loss:.4f}, saving model.")
    else:
        epochs_without_improvement += 1
        print(f"Epoch {epoch+1}/{num_epochs}, Loss did not improve. Consecutive epochs without improvement: {epochs_without_improvement}")

    if epochs_without_improvement >= patience:
        print("Early stopping triggered.")
        break

    end_time = time.time()
    elapsed_time = end_time - start_time
    remaining_time = elapsed_time * (num_epochs - epoch - 1)
    print(f"Epoch {epoch+1} completed in {elapsed_time:.2f}s, estimated time remaining: {remaining_time:.2f}s")

print("Training complete.")


Resolving data files:   0%|          | 0/298 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

trainable params: 2,621,440 || all params: 3,747,301,376 || trainable%: 0.0700
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-5-135575f02dc3>:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


已載入之前的模型權重，繼續訓練。
Epoch 1/15, Loss improved to 0.5547, saving model.
Epoch 1 completed in 189.29s, estimated time remaining: 2650.12s
Epoch 2/15, Loss improved to 0.5292, saving model.
Epoch 2 completed in 197.44s, estimated time remaining: 2566.69s
Epoch 3/15, Loss improved to 0.5156, saving model.
Epoch 3 completed in 199.63s, estimated time remaining: 2395.57s
Epoch 4/15, Loss improved to 0.4977, saving model.
Epoch 4 completed in 199.70s, estimated time remaining: 2196.67s
Epoch 5/15, Loss improved to 0.4843, saving model.
Epoch 5 completed in 197.39s, estimated time remaining: 1973.85s
Epoch 6/15, Loss improved to 0.4763, saving model.
Epoch 6 completed in 196.32s, estimated time remaining: 1766.87s
Epoch 7/15, Loss improved to 0.4607, saving model.
Epoch 7 completed in 201.81s, estimated time remaining: 1614.51s
Epoch 8/15, Loss improved to 0.4517, saving model.
Epoch 8 completed in 205.99s, estimated time remaining: 1441.91s
Epoch 9/15, Loss improved to 0.4446, saving model.
Epoc

# 附贈 正樣本訓練

In [ ]:
# @title
import os
import shutil
import pandas as pd
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
import time

# 1. 準備資料集
# 定義路徑
root_dir = '/content/'  # 資料集根目錄
output_dir = '/content/'  # 輸出資料集目錄

# 建立輸出目錄
os.makedirs(output_dir, exist_ok=True)

# 處理資料集並建立 metadata.csv
metadata = pd.DataFrame(columns=['file_name', 'additional_feature'])
data_to_append = []

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    print(f"Processing subdirectory: {subdir_path}")

    if os.path.isdir(subdir_path):
        image_file = None
        text = None

        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)

            if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_file = file
                dest_path = os.path.join(output_dir, image_file)
                if not os.path.exists(dest_path):
                    shutil.copy(file_path, output_dir)
                    print(f"Copied {file} to {output_dir}")
                else:
                    print(f"Image {file} already exists in {output_dir}, skipping copy.")

            elif file.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()

        if image_file and text:
            data_to_append.append({'file_name': image_file, 'additional_feature': text})

metadata = pd.concat([metadata, pd.DataFrame(data_to_append)], ignore_index=True)
metadata.to_csv(f'{output_dir}/metadata.csv', index=False, encoding='utf-8')
print("Images and text files processed and metadata saved successfully!")

# 使用 Hugging Face 的 ImageFolder
dataset = load_dataset("imagefolder", data_dir='/content/', split="train")
'''
NOTE:
load_dataset 是 Hugging Face Datasets 庫中的一個函數，用於載入各種類型的資料集。
"imagefolder"表示資料集是按圖像檔案夾結構組織的，會自動將每個子資料夾名稱當作標籤來建立標註資料
data_dir='/content/chiikawa'：指定資料集的根目錄。load_dataset 會在該目錄下搜尋所有子資料夾和圖像檔案
split="train"：指定要載入的資料集劃分。在 "imagefolder" 格式中，可以直接使用 split="train" 來表示載入整個train資料夾
'''

# 2. 定義 Dataset 和 DataLoader
# 將每筆圖像和文字描述資料轉換為模型可以接受的格式
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["additional_feature"]
        return encoding

# 將圖像和文字描述進行批次化處理，方便用於深度學習模型的訓練或推理
def collate_fn(batch):
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

# 3. 初始化模型和訓練參數
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)

# 定義 LoraConfig 和 get_peft_model 來微調模型
config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# 建立訓練資料集和 DataLoader
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collate_fn)

# 4. 開始訓練
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"

# 掛載 Google Drive
drive.mount('/content/drive')

# 建立模型儲存路徑
folder_path = '/content/drive/MyDrive/BLIP2_model'
os.makedirs(folder_path, exist_ok=True)
save_path = '/content/drive/MyDrive/BLIP2_model/best_blip2_model.pth'

# 訓練迴圈
num_epochs = 25
patience = 5  # 早停參數
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    start_time = time.time()
    epoch_loss = 0.0
    model.train()

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_epoch_loss = epoch_loss / len(train_dataloader)

    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), save_path)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss improved to {avg_epoch_loss:.4f}, saving model.")
    else:
        epochs_without_improvement += 1
        print(f"Epoch {epoch+1}/{num_epochs}, Loss did not improve. Consecutive epochs without improvement: {epochs_without_improvement}")

    if epochs_without_improvement >= patience:
        print("Early stopping triggered.")
        break

    end_time = time.time()
    elapsed_time = end_time - start_time
    remaining_time = elapsed_time * (num_epochs - epoch - 1)
    print(f"Epoch {epoch+1} completed in {elapsed_time:.2f}s, estimated time remaining: {remaining_time:.2f}s")

print("Training complete.")

# 不滿意繼續訓練

In [ ]:
# @title
import os
import shutil
import pandas as pd
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from transformers import AutoProcessor, Blip2ForConditionalGeneration
from peft import LoraConfig, get_peft_model
import torch
from google.colab import drive
import time

# 1. 準備資料集
root_dir = '/content/'  # 資料集根目錄
output_dir = '/content/'  # 輸出資料集目錄

os.makedirs(output_dir, exist_ok=True)

# 處理資料集並建立 metadata.csv
metadata = pd.DataFrame(columns=['file_name', 'additional_feature'])
data_to_append = []

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        image_file = None
        text = None
        for file in os.listdir(subdir_path):
            file_path = os.path.join(subdir_path, file)
            if file.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_file = file
                dest_path = os.path.join(output_dir, image_file)
                if not os.path.exists(dest_path):
                    shutil.copy(file_path, output_dir)
            elif file.endswith('.txt'):
                with open(file_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()
        if image_file and text:
            data_to_append.append({'file_name': image_file, 'additional_feature': text})

metadata = pd.concat([metadata, pd.DataFrame(data_to_append)], ignore_index=True)
metadata.to_csv(f'{output_dir}/metadata.csv', index=False, encoding='utf-8')


dataset = load_dataset("imagefolder", data_dir='/content/', split="train")

# 2. 定義 Dataset 和 DataLoader
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        encoding = self.processor(images=item["image"], padding="max_length", return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = item["additional_feature"]
        return encoding

def collate_fn(batch):
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

# 3. 初始化模型和訓練參數
processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("ybelkada/blip2-opt-2.7b-fp16-sharded", device_map="auto", load_in_8bit=True)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# 檢查是否有已保存的模型
save_path = '/content/drive/MyDrive/BLIP2_model/best_blip2_model.pth'
if os.path.exists(save_path):
    model.load_state_dict(torch.load(save_path))
    print("Loaded previously saved model.")

# 建立訓練資料集和 DataLoader
train_dataset = ImageCaptioningDataset(dataset, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2, collate_fn=collate_fn)

# 4. 開始訓練
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
device = "cuda" if torch.cuda.is_available() else "cpu"
drive.mount('/content/drive')

# 訓練迴圈
num_epochs = 30
patience = 5
best_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(num_epochs):
    start_time = time.time()
    epoch_loss = 0.0
    model.train()

    for batch in train_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        optimizer.zero_grad()
        outputs = model(pixel_values=pixel_values, input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_epoch_loss = epoch_loss / len(train_dataloader)

    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), save_path)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss improved to {avg_epoch_loss:.4f}, saving model.")
    else:
        epochs_without_improvement += 1
        print(f"Epoch {epoch+1}/{num_epochs}, Loss did not improve. Consecutive epochs without improvement: {epochs_without_improvement}")

    if epochs_without_improvement >= patience:
        print("Early stopping triggered.")
        break

    end_time = time.time()
    elapsed_time = end_time - start_time
    remaining_time = elapsed_time * (num_epochs - epoch - 1)
    print(f"Epoch {epoch+1} completed in {elapsed_time:.2f}s, estimated time remaining: {remaining_time:.2f}s")

print("Training complete.")


Resolving data files:   0%|          | 0/298 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

...

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/782M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

trainable params: 2,621,440 || all params: 3,747,301,376 || trainable%: 0.0700


<ipython-input-5-efeab33a974f>:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


Loaded previously saved model.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/30, Loss improved to 0.4453, saving model.
Epoch 1 completed in 197.42s, estimated time remaining: 5725.21s
Epoch 2/30, Loss improved to 0.4355, saving model.
Epoch 2 completed in 213.65s, estimated time remaining: 5982.31s
Epoch 3/30, Loss improved to 0.4285, saving model.
Epoch 3 completed in 208.72s, estimated time remaining: 5635.49s
Epoch 4/30, Loss improved to 0.4230, saving model.
Epoch 4 completed in 215.64s, estimated time remaining: 5606.62s
Epoch 5/30, Loss improved to 0.4194, saving model.
Epoch 5 completed in 214.06s, estimated time remaining: 5351.61s
Epoch 6/30, Loss did not improve. Consecutive epochs without improvement: 1
Epoch 6 completed in 189.58s, estimated time remaining: 4549.89s
Epoch 7/30, Loss improved to 0.4123, saving model.
Epoch 7 completed in 196.62s, estimated time remaining: 4522.29s
Epo

## 完成 灑花 🖖